# Esercitazione 2

Sempre partendo dai dati sulle definizioni, si richiede di provare a costruire un sistema che utilizzi la molteplicità delle definizioni per risalire al termine "target" in maniera automatica. Non si richiede di "indovinare" ogni termine, ma di avvicinarsi (almeno semanticamente) alla risposta. Provare più soluzioni, includendo meccanismi di filtro delle definizioni (ad es. escludendo quelle meno informative o con caratteristiche particolari), di ricerca nell'albero tassonomico di WordNet (provando a partire da candidati "genus", secondo il principio Genus-Differentia), ecc.

In [247]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

## Metodi di supporto

### Normalizzazione della frase

In [248]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def normalize(sentence):
    tokens = nltk.word_tokenize(sentence)

    tokens = [token for token in tokens if token not in string.punctuation] #tolgo la punteggiatura
    tokens = [token.lower() for token in tokens] # sostituisco le maiuscole con le minuscole
    tokens = [token for token in tokens if token not in stop_words] # rimuovo le stop words
    tokens = [lemmatizer.lemmatize(token) for token in tokens] # lemmatizzo
    
    return tokens

In [249]:
def get_nouns(tokens):
    #mantengo solo i sostantivi
    tokens = nltk.pos_tag(tokens)
    tokens = [word for word in tokens if word[1] in ['NN', 'NNS', 'NNP', 'NNPS']]
    return [word[0] for word in tokens]

### Algoritmo Simplified Lesk

In [250]:
def get_context(definitions):
    context = set()
    for definition in definitions:
        context = context.union(set(definition))
    return context

def simplified_lesk(word, context):
    best_sense = None
    max_overlap = 0

    for sense in wn.synsets(word, pos='n'):
        signature = set(normalize(sense.definition())).union(set(normalize(' '.join(sense.examples()))))
        overlap = len(context.intersection(signature))
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense
    
    return best_sense


### Ottenimento dei genus

In [251]:
'''
Restituisce come genus i token che compaiono più frequentemente nelle definizioni
'''
def get_genus(definitions, number_genus=3):
    words = []
    for definition in definitions:
        words += get_nouns(definition)

    return [tupla[0] for tupla in nltk.FreqDist(words).most_common(number_genus)]

## Approccio 1: recursive simplified lesk

Partendo dal genus, si prendono tutti i suoi synset, e ad ognuno di essi si applica il recursive_simplified_lesk. La caratteristica
di questo algoritmo è che non si ferma al primo livello di iponimi del genus, ma per ognuno di essi scende di un certo numero di livelli (iperparametro da specificare). Così facendo vengono analizzati un numero maggiore di synsets così da avere più probabilità di ottenere il synset corretto.

In [252]:
'''
Variante ricorsiva del lesk, che permette di scendere di livello nella
gerarchia del genus
'''
def recursive_simplified_lesk(level, senses, current_syn, context):
    if level != 0:
        for syn in current_syn.hyponyms():
            branch_senses = []
            signature = set(normalize(syn.definition())).union(normalize(' ' .join(syn.examples())))
            overlap = len(context.intersection(signature))
            branch_senses.append((syn,overlap))
            recursive_simplified_lesk(level - 1, branch_senses, syn, context)
            senses = senses + branch_senses
        return senses

'''
Metodo per predire il token dato un insieme di definizioni
'''
def predict_token(definitions):
    ret = {}
    genus = get_genus(definitions)
    # ottengo il contesto
    context = get_context(definitions)

    for gen in genus:
        for syn_genus in wn.synsets(gen, pos='n')[0:3]:
            senses = []
            ret[syn_genus] = recursive_simplified_lesk(3, senses, syn_genus, context)

    array_concatenato = []
    for array in ret.values():
        array_concatenato.extend(array)
    return sorted(array_concatenato, key=lambda tupla: tupla[1], reverse=True)[0:5]

In [253]:
corpus1 = pd.read_csv('definizioni.tsv', sep='\t', engine='python')
# remove sentence too long or too short
# Calcolo dei percentili per determinare i limiti
q5 = corpus1.apply(lambda colonna: colonna.apply(len)).quantile(0.05)
q95 = corpus1.apply(lambda colonna: colonna.apply(len)).quantile(0.95)
# Sostituzione delle stringhe con la stringa vuota
corpus1 = corpus1.apply(lambda colonna: colonna if colonna.dtype != object else np.where((colonna.str.len() < q5[colonna.name]) | (colonna.str.len() > q95[colonna.name]), "", colonna))

corpus1['door'] = corpus1['door'].apply(normalize)
corpus1['ladybug'] = corpus1['ladybug'].apply(normalize)
corpus1['pain'] = corpus1['pain'].apply(normalize)
corpus1['blurriness'] = corpus1['blurriness'].apply(normalize)

for token in corpus1.columns:
    print('token:', token)
    predicted_token = predict_token(corpus1[token])
    print(predicted_token)
    print("\n")

token: door
[(Synset('opening.n.01'), 6), (Synset('entrance.n.01'), 5), (Synset('compartment.n.02'), 4), (Synset('study.n.05'), 4), (Synset('headroom.n.01'), 4)]


token: ladybug
[(Synset('dipterous_insect.n.01'), 4), (Synset('leaf_miner.n.01'), 3), (Synset('lepidopterous_insect.n.01'), 3), (Synset('psocopterous_insect.n.01'), 3), (Synset('web_spinner.n.01'), 3)]


token: pain
[(Synset('taste.n.01'), 3), (Synset('vision.n.03'), 3), (Synset('masking.n.02'), 2), (Synset('smell.n.01'), 2), (Synset('sound.n.02'), 2)]


token: blurriness
[(Synset('collage.n.01'), 6), (Synset('naked_eye.n.01'), 4), (Synset('mental_picture.n.01'), 3), (Synset('visual_image.n.01'), 3), (Synset('iconography.n.01'), 3)]


